In [6]:
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from math import exp
from math import log

def parseData(fname):
    for l in urlopen(fname):
        yield eval(l)



In [7]:
print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("done")

Reading data...
done


In [8]:
## random data
random.shuffle(data)

## split
training_data= data[:int(len(data)/3)]
vali_data= data[int(len(data)/3):2*int(len(data)/3)]
test_data= data[2*int(len(data)/3):]

In [9]:
def feature(datum):
  feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
  return feat

X_train = [feature(d) for d in training_data]
y_train = [d['beer/ABV'] >= 6.5 for d in training_data]

X_vali = [feature(d) for d in vali_data]
y_vali = [d['beer/ABV'] >= 6.5 for d in vali_data]

X_test = [feature(d) for d in test_data]
y_test = [d['beer/ABV'] >= 6.5 for d in test_data]


def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

In [10]:
##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

##################################################
# Train                                          #
##################################################

def train(lam):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X_train[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
  return theta

##################################################
# Predict                                        #
##################################################

def performance(theta, X, y):
  scores = [inner(theta,x) for x in X]
  predictions = [s > 0 for s in scores]
  correct = [(a==b) for (a,b) in zip(predictions,y)]
  acc = sum(correct) * 1.0 / len(correct)
  return acc

In [11]:
##################################################
# Validation pipeline                            #
##################################################

lam = 1.0

theta = train(lam)
acc = performance(theta, X_train, y_train)
print("lambda = " + str(lam) + ":\taccuracy of train set=" + str(acc))

acc = performance(theta, X_vali, y_vali)
print("lambda = " + str(lam) + ":\taccuracy of validation set=" + str(acc))

acc = performance(theta, X_test, y_test)
print("lambda = " + str(lam) + ":\taccuracy of test set=" + str(acc))

lambda = 1.0:	accuracy of train set=0.7170886835473419
lambda = 1.0:	accuracy of validation set=0.7157086283451338
lambda = 1.0:	accuracy of test set=0.7220422366210704


In [19]:
a=[True]
if a[0] ==True :
    print('1')


1
